# ParikhSamuolisReclassificationNN Final Project File 3
## date last modified: December 7, 2024
### how to link to github --> https://saturncloud.io/blog/how-to-add-jupyter-notebook-to-github/

# Loading images and necessary functions - run each time

In [1]:
import os
from PIL import Image
import numpy as np
import random
import shutil

# define directories for train and validation sets
root_dir = '/projectnb/ds340/projects/Samuolis_Parikh_Image_Data/'

train_dir = root_dir +"resized_images/train"
validation_dir = root_dir + "resized_images/validation"

train_target = train_dir +"/baldeagle"
train_nontarget = train_dir +"/nonbaldeagle"

val_target = validation_dir +"/baldeagle"
val_nontarget = validation_dir +"/nonbaldeagle"

def load_images_from_folders(folder1, folder2, img_size = (224,224)):
    images = []
    labels = []
    
    # load images from the first folder
    for filename in os.listdir(folder1):
        img_path = os.path.join(folder1, filename)
        try:
            with Image.open(img_path) as img:
                img = img.convert('RGB')
                img = img.resize(img_size)
                images.append(np.array(img))  # convert image to array
                labels.append(1)  # class label for folder1
        except Exception as e:
            print(f"Could not load image {filename} from {folder1}: {e}")

    # load images from the second folder
    for filename in os.listdir(folder2):
        img_path = os.path.join(folder2, filename)
        try:
            with Image.open(img_path) as img:
                img = img.convert('RGB')
                img = img.resize(img_size)
                images.append(np.array(img))
                labels.append(0)  # class label for folder2
        except Exception as e:
            print(f"Could not load image {filename} from {folder2}: {e}")

    # convert lists to NumPy arrays
    
    images = np.array(images)
    labels = np.array(labels)
    
    return images, labels

images_train, label_train = load_images_from_folders(train_target, train_nontarget)
images_val, label_val = load_images_from_folders(val_target, val_nontarget)

In [2]:
## for debugging:
print(images_train.shape, label_train.shape, type(images_train))
print(images_train.min(), images_train.max())  # expected: 0 255, later will normalize
print(f"Initial eagle count: {np.sum(label_train == 1)}")
print(f"Initial noneagle count: {np.sum(label_train == 0)}")

(5200, 224, 224, 3) (5200,) <class 'numpy.ndarray'>
0 255
Initial eagle count: 1300
Initial noneagle count: 3900


In [3]:
def change_labels(labels, percentage):
    random.seed(340)
    label_one_indices = np.where(labels == 1)[0]
    
    n = int(len(label_one_indices) * (percentage / 100))
    
    indices_to_change = np.random.choice(label_one_indices, size=n, replace=False)
    
    labels[indices_to_change] = 0
    
    return labels, indices_to_change

# for example, change 20% of label 1s to label 0
percentage = 0  
# changed_indices
# label_train, changed_indices = change_labels(label_train, percentage)

In [4]:
import os
import warnings
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# suppress TensorFlow and CUDA logs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # show only errors
warnings.filterwarnings('ignore', category=UserWarning)

# configure Absl logging
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

# set deterministic operations and random seed
tf.keras.utils.set_random_seed(340)
tf.config.experimental.enable_op_determinism()

2024-12-07 17:26:24.507485: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-07 17:26:24.521420: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-07 17:26:24.525495: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
]
epochs = 15
# restore best weights make the model be the one that was the best instead of last one
# patience changed from 4-->3

In [6]:
# reload images
images_train, label_train = load_images_from_folders(train_target, train_nontarget)
percentage = 20  
# changed_indices
label_train, changed_indices = change_labels(label_train, percentage)

In [7]:
## for debugging:
print(f"New eagle count: {np.sum(label_train == 1)}")
print(f"New noneagle count: {np.sum(label_train == 0)}")

New eagle count: 1040
New noneagle count: 4160


In [8]:
from tensorflow.keras.layers import Input, Dropout, Concatenate
confidence_init = confidence_init = np.array([.35 if x<.5 else 1 for x in label_train]).reshape(5200,1)
# start with all 1s for confidence
# this doesn't work --- we don't know before hand which indices we aren't confident about, we especially don't know 
# to specifically be less confident for the labels that we changed
# confidence_init[label_train == 0] = 0  # Set confidence to 0 for original 0 labels
# confidence_init[changed_indices] = 0.35  # Set confidence to 0.35 for flipped labels
# confidence_init = confidence_init.reshape(-1, 1)  # Reshape to (N, 1)

# print data statistics
print(f"New eagle count: {np.sum(label_train == 1)}")
print(f"New noneagle count: {np.sum(label_train == 0)}")
print(f"Confidence values: {confidence_init[:10].flatten()}")

New eagle count: 1040
New noneagle count: 4160
Confidence values: [0.35 1.   1.   1.   1.   1.   1.   0.35 1.   1.  ]


In [9]:
# remake models
# we have full confidence if it is a 1, the lower the number the more confident you are in the 0 class -- .999999 vs .00004

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = True
    

# add new fully connected layers for binary classification
image_input = base_model.input
x = base_model.output
x = Flatten()(x)

additional_input = Input(shape=(1,), name="additional_input") 
y = Dense(64, activation='relu')(additional_input) 
y = Dropout(0.1)(y) 

combined = Concatenate()([x, y]) # 2 channels
combined = Dense(256, activation='relu')(combined)
combined = Dense(1, activation='sigmoid')(combined) 

model = Model(inputs=[image_input, additional_input], outputs=combined)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'], jit_compile=False)

In [10]:
iteration_list = [] # update done in code
adjusted_indices_list = [] # update done in code
avg_confidences_list = [] # update done in code
totalnumeagles_modelbelievesnoteagles = [] # update done in code
totalnumimages_modelbelievesnoteagles = [] # update done in code
change_ratio_list = [] # update done in code
more_confident_list = [] # update done in code


n_percentage = 5  # % of least confident eagle predictions to adjust
max_iterations = 10 
convergence_tolerance = 0  # stop if change ratio is this
high_conf_threshold = 0.8  # threshold for confident eagle flips
confidence_init = confidence_init = np.array([.35 if x<.5 else 1 for x in label_train]).reshape(5200,1)

num_incorrectly_modified = 0
num_modified = 0
for iteration in range(max_iterations):
    print(f"Iteration {iteration + 1}...")
    iteration_list.append(iteration+1) # updating the lists

    # step 1: train model
    history = model.fit(
        [images_train, confidence_init],
        label_train,
        batch_size=100, # ok this might seem crazy but im wondering if w batch=32 it wasn't encountering enough wrong labels 
        epochs=1,
        # validation_data=([images_val, np.ones((len(images_val), 1))], label_val), # this was a line from chat, replaced w ours instead below
        validation_data=([images_val, label_val.reshape(-1,1)], label_val),
        callbacks=callbacks,
        shuffle = True,
        verbose=0
    )

    # step 2: predict probabilities
    preds = model.predict([images_train, confidence_init]).flatten()

    # step 3: identify least confident eagle predictions
    low_confidence_indices = np.where((label_train == 0) & (preds < 0.5) & (preds != 0))[0] # grabbing indices where label_train is 0 (noneagle), focusing in on the misclassified
    filtered_indices = low_confidence_indices[confidence_init.flatten()[low_confidence_indices]!= 0]
    sorted_indices = filtered_indices[np.argsort(preds[filtered_indices])] # sorts the preds low to high
    to_adjust = sorted_indices[:int(len(sorted_indices) * (n_percentage / 100))] # only grabbing 5% rn of the bottom
    

    # step 4: update confidence for least confident predictions
    if len(to_adjust) > 0:
        confidence_init[to_adjust] = 0  # reduce confidence to 0 for the indices we picked by %
        avg_confidence = np.mean(preds[to_adjust])
        print(f"Adjusted {len(to_adjust)} indices, avg confidence: {avg_confidence:.4f}")
        adjusted_indices_list.append(len(to_adjust)) # updating the lists
        avg_confidences_list.append(round(avg_confidence,4)) # updating the lists
        
    else:
        print("No indices to adjust in this iteration.")
        
    print(f"Changed Indices: {sorted(to_adjust)}")
    print(f"Predictions for Changed Indices: {preds[to_adjust]}")
    wrongly_switched = [x for x in changed_indices if x in to_adjust]
    print("FOR CHECKING -- THE EAGLES WHICH THE MODEL GOT EVEN MORE CONFIDENT WASN'T EAGLES:", wrongly_switched)
    more_confident_list.append(len(wrongly_switched)) # updating the lists
    num_incorrectly_modified += len(wrongly_switched)
    num_modified += len(to_adjust)
    print("Total Number of Eagles the model believes are not eagles", num_incorrectly_modified)
    totalnumeagles_modelbelievesnoteagles.append(num_incorrectly_modified) # updating the lists
    print("Total Number of images the model believes are not eagles", num_modified)
    totalnumimages_modelbelievesnoteagles.append(num_modified) # updating the lists
 

Iteration 1...


W0000 00:00:1733610396.290729 1984735 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733610396.327169 1984735 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733610396.374559 1984735 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733610396.387045 1984735 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733610396.413693 1984735 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733610396.439054 1984735 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733610396.458536 1984735 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733610396.495940 1984735 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733610396.512662 1984735 gp

  4/163 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step   

W0000 00:00:1733610421.689047 1984735 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733610421.692358 1984735 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733610421.699960 1984735 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733610421.709184 1984735 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733610421.716792 1984735 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733610421.723981 1984735 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733610421.729888 1984735 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733610421.738908 1984735 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733610421.743928 1984735 gp

163/163 ━━━━━━━━━━━━━━━━━━━━ 7s 41ms/step


W0000 00:00:1733610428.390992 1984735 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733610428.393910 1984735 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733610428.398697 1984735 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733610428.403472 1984735 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733610428.407838 1984735 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733610428.411543 1984735 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733610428.415364 1984735 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733610428.419994 1984735 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733610428.423740 1984735 gp

Adjusted 208 indices, avg confidence: 0.1062
Changed Indices: [15, 91, 306, 329, 533, 564, 578, 666, 684, 803, 837, 1089, 1109, 1148, 1217, 1300, 1302, 1321, 1330, 1334, 1349, 1355, 1430, 1454, 1463, 1528, 1559, 1562, 1636, 1665, 1666, 1689, 1800, 1810, 1812, 1815, 1821, 1830, 1842, 1870, 1880, 1882, 1922, 1932, 1937, 1951, 1999, 2017, 2037, 2047, 2075, 2095, 2108, 2122, 2126, 2145, 2170, 2171, 2209, 2224, 2287, 2325, 2338, 2340, 2352, 2367, 2383, 2395, 2410, 2417, 2433, 2443, 2447, 2459, 2475, 2479, 2498, 2499, 2516, 2528, 2556, 2570, 2611, 2617, 2637, 2661, 2715, 2740, 2741, 2743, 2744, 2760, 2768, 2777, 2778, 2798, 2805, 2811, 2825, 2829, 2855, 2891, 2969, 3026, 3034, 3104, 3113, 3130, 3145, 3154, 3202, 3231, 3315, 3338, 3344, 3356, 3362, 3377, 3387, 3395, 3400, 3433, 3461, 3477, 3497, 3538, 3552, 3563, 3582, 3634, 3677, 3700, 3701, 3714, 3728, 3755, 3757, 3761, 3775, 3797, 3799, 3812, 3841, 3875, 3885, 3890, 3936, 3956, 3969, 4002, 4005, 4021, 4031, 4050, 4075, 4099, 4101, 4114, 41

In [11]:
def top_n_percent_indices(predictions, n_percent):

    target_indices = np.where(label_train == 0)[0]
    filtered_preds = predictions[target_indices]
    
    # calculate the number of top elements to select
    num_top_elements = int(np.ceil(len(filtered_preds) * n_percent / 100))
    
    # get the indices of the sorted values (descending order)
    sorted_indices = np.argsort(filtered_preds)[::-1]
    
    # select the top n_percent indices
    top_indices = sorted_indices[:num_top_elements]

    top_original_indices = target_indices[top_indices]
    
    return top_original_indices
n_percent = 5
high_confidence_indices = top_n_percent_indices(preds,n_percent)
high_confidence_indices
actually_eagles = [x for x in high_confidence_indices if x in changed_indices]
# print("The least confident not-eagles that are actually eagles:", actually_eagles)
print(f"How many actual eagles in top {n_percent}% is: {len(actually_eagles)}")
print(f"How many total are in the top {n_percent}% is: {len(high_confidence_indices)}")

How many actual eagles in top 5% is: 60
How many total are in the top 5% is: 208


### This is now simulating manually looking through the top n% of "non-eagles" to change their label. In the real application you would have someone actually looking at each of these images and checking if they were eagles, but since in this training we just know the which indices are actually eagles, we're just going to change it automatically. The only assumption is that the human is able to actually correctly find all the eagles when actually looking at them. 

In [12]:
label_train[actually_eagles] = 1

data_gen_args = {
    'rescale': 1.0 / 255.0,  # normalize pixel values
}

train_datagen = ImageDataGenerator(**data_gen_args)
validation_datagen = ImageDataGenerator(**data_gen_args)


validation_generator = validation_datagen.flow(
    x = images_val,
    y = label_val,
    batch_size=32,
)

In [13]:
# trying moderate instead bc 12/7 pass showed better results after changes i made - riya
augmented_data_gen_args = {
    'rescale': 1.0 / 255.0,
    'horizontal_flip': True,
    'rotation_range': 20,
    'width_shift_range': 0.2,  # shift images horizontally
    'height_shift_range': 0.2,  # shift images vertically
    'brightness_range': [0.8, 1.2],  # adjust brightness between 80% and 120%
    'zoom_range': 0.2,
}

train_augmented_datagen = ImageDataGenerator(**augmented_data_gen_args)

train_augmented_generator = train_augmented_datagen.flow(
    x=images_train,
    y=label_train,
    batch_size=32,
)

In [14]:
final_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in final_model.layers:
    # layer.trainable = True tomas, riya change below
    layer.trainable = False

# add new fully connected layers for binary classification
x = final_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)  # sigmoid for binary 

model = Model(inputs=final_model.input, outputs=x)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'], jit_compile=False)

In [15]:
from sklearn.metrics import confusion_matrix, classification_report

callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
]

print("Simple Augmentation") 
# side note from riya: i forgot to change this label to moderate augmentation, but yes, we are using moderate here and not simple. 
# i am not going to rerun anything because this is taking too long to run anyways. hope you all understand. 
# please do. this is my 3rd SCC request of the day and i will scream in mugar if i have to request a fourth.
print()
history_simple = model.fit(
    train_augmented_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks = callbacks
)

# confusion matrix and classification report
val_preds = (model.predict(images_val) > 0.5).astype(int).flatten()
conf_matrix = confusion_matrix(label_val, val_preds)
class_report = classification_report(label_val, val_preds, target_names=["Not Eagle", "Eagle"])

print("\nConfusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", class_report)

Simple Augmentation

Epoch 1/15
163/163 ━━━━━━━━━━━━━━━━━━━━ 54s 312ms/step - accuracy: 0.7772 - loss: 1.1066 - val_accuracy: 0.8600 - val_loss: 0.3302
Epoch 2/15


W0000 00:00:1733610764.324044 1984735 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733610764.326811 1984735 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733610764.330291 1984735 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733610764.333231 1984735 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733610764.336709 1984735 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733610764.339622 1984735 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733610764.343058 1984735 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733610764.346796 1984735 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733610764.350488 1984735 gp

163/163 ━━━━━━━━━━━━━━━━━━━━ 52s 309ms/step - accuracy: 0.8623 - loss: 0.3309 - val_accuracy: 0.8900 - val_loss: 0.2287
Epoch 3/15
163/163 ━━━━━━━━━━━━━━━━━━━━ 52s 309ms/step - accuracy: 0.8917 - loss: 0.2569 - val_accuracy: 0.9300 - val_loss: 0.1565
Epoch 4/15
163/163 ━━━━━━━━━━━━━━━━━━━━ 52s 310ms/step - accuracy: 0.9012 - loss: 0.2350 - val_accuracy: 0.9450 - val_loss: 0.1400
Epoch 5/15
163/163 ━━━━━━━━━━━━━━━━━━━━ 52s 310ms/step - accuracy: 0.9054 - loss: 0.2246 - val_accuracy: 0.9550 - val_loss: 0.1280
Epoch 6/15
163/163 ━━━━━━━━━━━━━━━━━━━━ 52s 309ms/step - accuracy: 0.9150 - loss: 0.2065 - val_accuracy: 0.9500 - val_loss: 0.1313
Epoch 7/15
163/163 ━━━━━━━━━━━━━━━━━━━━ 52s 308ms/step - accuracy: 0.8942 - loss: 0.2463 - val_accuracy: 0.9550 - val_loss: 0.1201
Epoch 8/15
163/163 ━━━━━━━━━━━━━━━━━━━━ 52s 309ms/step - accuracy: 0.9141 - loss: 0.1915 - val_accuracy: 0.9550 - val_loss: 0.1240
Epoch 9/15
163/163 ━━━━━━━━━━━━━━━━━━━━ 52s 309ms/step - accuracy: 0.9172 - loss: 0.1836 - val

# Comparisons !!!!

*We will only be focusing on the moderate augmentations.*

Misclassified, no augmentation: 
- Average Validation Accuracy: 93.3%
- Average Validation Loss: 0.1476

Misclassified, moderate augmentation: 
- Average Validation Accuracy:  93.5%
- Average Validation Loss: 0.1646

Misclassified, reclassified, moderate augmentation: above result: 
- Average Validation Accuracy: 94.80% 
- Average Validation Loss: 0.1513

# Final Conclusions

**1. Precision, Recall, F1-Score, and Support**
  
*Precision -  accuracy of positive predictions*
- For "Not Eagle" class (Label = 0): Precision: 0.97
- This means that when the model predicts "Not Eagle", 97% of the time, the prediction is correct. It reflects the accuracy of positive predictions.
- For "Eagle" class (Label = 1): Precision: 0.85
- This means that when the model predicts "Eagle", 85% of the time, the prediction is correct. The model is less precise compared to "Not Eagle" predictions.

*Recall - measures the ability of the model to find all the relevant instances*
- For "Not Eagle" class (Label = 0): Recall: 0.95
- This indicates that out of all actual "Not Eagle" instances, 95% were correctly identified by the model. 
- For "Eagle" class (Label = 1): Recall: 0.92
- This indicates that 92% of the actual "Eagle" instances were correctly identified. The model does well in recognizing "Eagle" instances but is slightly less precise in its predictions compared to the "Not Eagle" class.

*F1 Score - harmonic mean of precision and recall, giving a balanced measure of the model's performance*
- For "Not Eagle" class (Label = 0): F1-Score: 0.96
- An F1-Score of 0.96 is strong, indicating a good balance between precision and recall for this class.
- For "Eagle" class (Label = 1): F1-Score: 0.88
- Indicates a reasonable balance of precision and recall, though the precision is slightly lower than that for "Not Eagle".

**2. Overall Metrics**

*Accuracy: 0.94*
The model correctly predicted the class (either "Not Eagle" or "Eagle") for 94% of all instances in the dataset.

*Weighted Average Precision, Recall, and F1 Score: 0.94*
The weighted precision takes into account the number of instances in each class, giving more weight to the performance on the larger class ("Not Eagle"). The weighted average precision is high, showing that overall precision is good.
Similarly, the weighted recall shows that the model is performing well in identifying both classes, considering their frequency in the dataset.
The weighted average F1-Score is also strong at 0.94, which reflects the model's overall good performance, considering both classes.